In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pickle
import warnings

from scipy import stats

# Ignore all warnings
warnings.filterwarnings('ignore')

In [3]:
full_analysis = False
if full_analysis: 
    with open('results_rob.pkl', 'rb') as f:
        results = pickle.load(f)
else:
    with open('results_rob_paper.pkl', 'rb') as f:
        results = pickle.load(f)

In [17]:
setup_names = {}
base_names = ['FCER', 'CCER', 'PFCER', 'PCCER']
norm_names = ['', ' Dist', ' Std', ' Abs', ' Var']
for i, base in enumerate(['ce', 'cce', 'pce', 'pcce']):
    for j, setup in enumerate(['', '_dist', '_std', '_abs', '_var']):
        setup_names[base + setup] = base_names[i] + norm_names[j]
setup_names

{'ce': 'FCER',
 'ce_dist': 'FCER Dist',
 'ce_std': 'FCER Std',
 'ce_abs': 'FCER Abs',
 'ce_var': 'FCER Var',
 'cce': 'CCER',
 'cce_dist': 'CCER Dist',
 'cce_std': 'CCER Std',
 'cce_abs': 'CCER Abs',
 'cce_var': 'CCER Var',
 'pce': 'PFCER',
 'pce_dist': 'PFCER Dist',
 'pce_std': 'PFCER Std',
 'pce_abs': 'PFCER Abs',
 'pce_var': 'PFCER Var',
 'pcce': 'PCCER',
 'pcce_dist': 'PCCER Dist',
 'pcce_std': 'PCCER Std',
 'pcce_abs': 'PCCER Abs',
 'pcce_var': 'PCCER Var'}

In [18]:
stab_timer = results['housing']['RF']['stab_timer']
rob_timer = results['housing']['RF']['rob_timer']

stab = 'Stability'
rob = 'Robustness'
timer = {}
timer[stab] = {}
timer[rob] = {}
for setup in stab_timer.keys():
    timer[stab][setup_names[setup]] = np.mean(stab_timer[setup]) / 10 # number of instances 
    timer[rob][setup_names[setup]] = np.mean(rob_timer[setup]) / 10 # number of instances 

timer = pd.DataFrame(timer)
timer

,Stability,Robustness
FCER,2.573499e-01,2.662939e-01
CCER,3.986868e-01,3.818672e-01
FCER Var,7.955051e-01,8.160611e-01
CCER Var,1.156076e+00,1.137427e+00
PFCER,6.246670e-01,6.406583e-01
PCCER,9.211112e-01,8.948543e-01
PFCER Var,1.185845e+00,1.204821e+00
PCCER Var,1.718070e+00,1.656689e+00


In [19]:
stability = results['housing']['RF']['stability']
robustness = results['housing']['RF']['robustness']

stab = 'Stability'
rob = 'Robustness'
performance = {}
performance[stab] = {}
performance[rob] = {}
performance['Prediction Variance'] = np.mean([np.var([robustness['predict'][i][j] for i in range(len(robustness['predict']))]) for j in range(len(robustness['predict'][0]))])
for setup in stability.keys():
    # Get the most important feature for each instance using the absolute value of the feature weights
    stab_feature = []
    rob_feature = []
    for j in range(len(stability[setup][0])): # number of instances        
        stab_feature.append([np.argmax(np.abs(stability[setup][i][j]['predict'])) for i in range(len(stability[setup]))])
        rob_feature.append([np.argmax(np.abs(robustness[setup][i][j]['predict'])) for i in range(len(robustness[setup]))])
    stab_feature = stats.mode(stab_feature, axis=1)
    rob_feature = stats.mode(rob_feature, axis=1)
    
    
    performance[stab][setup_names[setup]] = np.mean([np.var([stability[setup][i][j]['predict'][stab_feature.mode[j]] for i in range(len(stability[setup]))]) for j in range(len(stability[setup][0]))])
    performance[rob][setup_names[setup]] = np.mean([np.var([robustness[setup][i][j]['predict'][rob_feature.mode[j]] for i in range(len(robustness[setup]))]) for j in range(len(robustness[setup][0]))])

performance = pd.DataFrame(performance)
pd.options.display.float_format = "{:e}".format
performance

,Stability,Robustness,Prediction Variance
FCER,1.047706e-32,8.010681e-03,9.068915e-05
CCER,1.848893e-33,2.053705e-03,9.068915e-05
FCER Var,1.294225e-32,7.278605e-03,9.068915e-05
CCER Var,1.040002e-33,1.811707e-03,9.068915e-05
PFCER,2.543178e-03,3.600766e-02,9.068915e-05
PCCER,2.387989e-03,1.375770e-02,9.068915e-05
PFCER Var,1.821867e-03,3.044382e-02,9.068915e-05
PCCER Var,2.901287e-03,9.606847e-03,9.068915e-05
